<a href="https://colab.research.google.com/github/lihasarora/Hands-on-Machine-Learning-/blob/main/MSBA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd, numpy as np, seaborn as sn, matplotlib.pyplot as plt
from tqdm import tqdm
from sklearn.utils import shuffle

## Sampling dist of sample mean

In [ ]:
def gen_pop(num_samples:int)->np.array:
  return np.random.normal(0,10,num_samples)

def gen_samples(sample_size:int, pop:np.array)->np.array:
  return np.random.choice(pop,sample_size)

In [ ]:
population = gen_pop(10000)

In [ ]:
dict_ = {}
for num in [10,40,100]:
  dict_[num] = [gen_samples(num,population).mean() for i in range(1000)]

In [ ]:
plt.figure(figsize = (20,10))
for sample_size in list(dict_.keys()):
  sn.kdeplot(dict_[sample_size], label = f"{sample_size}")
plt.axvline(population.mean(), linestyle = 'dashed', color = 'black')
plt.legend()

## Central Limit Theoram

 - In the above example, we considered the population to be normal. What if we don't know the population disstribution or the population is not normally distributed

 - One can see that the as sample size increases, the sampling distribution of sample mean gets normally distributed

In [ ]:
pop  = np.random.exponential(scale = 30, size = 10000)

plt.figure(figsize = (15,5))
plt.subplot(1,2,1)
sn.kdeplot(pop, fill = True)
plt.title("Population is exponentially distributed")

plt.subplot(1,2,2)

dict_ = {}
for num in [10,40,100]:
  dict_[num] = [gen_samples(num,pop).mean() for i in range(100)]

for sample_size in list(dict_.keys()):
  sn.kdeplot(dict_[sample_size], label = f"{sample_size}")
plt.axvline(pop.mean(), linestyle = 'dashed', color = 'black')
plt.legend()

# Do we need the the samples to be normally distributed for one sample t test?

In [ ]:
np.random.seed(42)
pop_e = np.random.exponential(scale = 5, size = 10000)
pop_n  = np.random.normal(5,10,10000)
plot = sn.displot(pop_e, bins = 30, kde  = True)
plot = sn.displot(pop_n, bins = 30, kde  = True)
# title = plt.title("Exponential population")
print(f"{pop_e.mean()=:0.2f} , {pop_n.mean()=:0.2f} ")

In [ ]:
# H0: mean = 4.89 Ha: mean ne 4.89
from scipy.stats import ttest_1samp
# True = Reject
hyp_dict_n = {}
hyp_dict_e = {}

for sample_size in tqdm([10,15,30,50,100,150,300,500]):
  hyp_test = pd.Series([True if ttest_1samp(a = gen_samples(sample_size,pop_n), popmean = pop_n.mean(),alternative= 'two-sided')[1] < 0.05 else False for x in range(10000)])
  hyp_dict_n[sample_size] = hyp_test.value_counts(normalize = True)[True]

  hyp_test = pd.Series([True if ttest_1samp(a = gen_samples(sample_size,pop_e), popmean = pop_e.mean(),alternative= 'two-sided')[1] < 0.05 else False for x in range(10000)])
  hyp_dict_e[sample_size] = hyp_test.value_counts(normalize = True)[True]


In [ ]:
plt.figure(figsize = (20,5))
plt.plot(list(hyp_dict_n.keys()),list(hyp_dict_n.values()))
plt.plot(list(hyp_dict_e.keys()),list(hyp_dict_e.values()))
plt.axhline(0.05, linestyle = 'dashed', color = 'red')

# 2 sample T Test

In [ ]:
np.random.seed(42)
pop1 = np.random.normal(loc = 4, scale = 3, size = 10000)
pop2 = np.random.normal(loc = 5, scale = 2.6, size = 10000)
sn.kdeplot(pop1, fill = True, label = "N-1")
sn.kdeplot(pop2, fill = True, label = "N-2")
plt.legend()

In [ ]:
# sample from both the distributions
np.random.seed(42)
samp1 = np.random.choice(pop1, size = 1000)
samp2 = np.random.choice(pop2, size = 1000)
sn.kdeplot(samp1, fill = True, label = "N-1")
sn.kdeplot(samp2, fill = True, label = "N-2")
plt.legend()
print(f"samp1 : {samp1.mean()=:0.2} ; {np.var(samp1, ddof = 1)=:0.2} ")
print(f"samp2 : {samp2.mean()=:0.2} ; {np.var(samp2, ddof = 1)=:0.2} ")

I have to estimate $\mu_1 - \mu_2$ using  $\bar x_1 - \bar x_2$ 

In [ ]:
# Lets get the sigma pooled
pooled_var = (np.var(samp1,ddof = 1)*(len(samp1)-1) + np.var(samp2,ddof = 1)*(len(samp2)-1))/(len(samp2)+len(samp1)-2)
pooled_sd = np.sqrt(pooled_var)* np.sqrt((1/len(samp1))+ (1/len(samp2))) 

t_value = (np.mean(samp1) - np.mean(samp2))/pooled_sd
print(f"{t_value = }")

In [ ]:
from scipy.stats import ttest_ind
ttest_ind(samp1, samp2)

The $h_0:$ There is a difference in the means is 0 or there is no difference in the mean is rejected 

One can see that the simulated distribution of the difference of means is very close to the true distribution and hence when there is no difference in the means, the sampling distribution of the difference of means does follow a T-distribution

In [ ]:
# What if we generate populations such that Null hypothesis is true
sample_size = 100
np.random.seed(42)
pop1 = np.random.normal(loc = 4, scale = 3, size = 10000)
pop2 = np.random.normal(loc = 4, scale = 3, size = 10000)

samp1 = np.random.choice(pop1, size = sample_size)
samp2 = np.random.choice(pop2, size = sample_size)

pooled_var = (np.var(samp1,ddof = 1)*(len(samp1)-1) + np.var(samp2,ddof = 1)*(len(samp2)-1))/(len(samp2)+len(samp1)-2)
pooled_sd = np.sqrt(pooled_var)* np.sqrt((1/len(samp1))+ (1/len(samp2))) 

t_value = (np.mean(samp1) - np.mean(samp2))/pooled_sd
print(f"{t_value = }")

print(f"Result of TTest:{ttest_ind(samp1, samp2)}")
## The ttest fails to reject the null hypothesis

In [ ]:
## How does the distribution of difference in mean look? 
sn.kdeplot(np.random.standard_t(df = len(samp1)+ len(samp2)-2, size = 1000)*pooled_sd, fill = True)
## Lets get the true distribution as well
sn.kdeplot([gen_samples(sample_size,pop2).mean() - gen_samples(sample_size,pop1).mean() for x in range(2000)], fill = True, color = 'red')


Lets do a bootstrapping method to calculate the difference in means

In [ ]:
sample_size = 20
# create two populations
np.random.seed(42)
pop1 = np.random.normal(loc = 4, scale = 3, size = 10000)
pop2 = np.random.normal(loc = 5, scale = 2.6, size = 10000)

# sample 100 from each
samp1 = np.random.choice(pop1,sample_size)
samp2 = np.random.choice(pop2,sample_size)


# Store in df for tracking
df = shuffle(
    pd.concat(
    [
        pd.DataFrame({
    'population': ['one']*len(samp1),
    'sample': samp1
}),
        pd.DataFrame({
    'population': ['two']*len(samp2),
    'sample': samp2
})
    ]
)
)
df.head()

In [ ]:
# conventional ttest
ttest_ind(samp1,samp2)

In [ ]:
## How does the difference in means look if we mix them 
## H0: Both the populations are same
np.random.seed(42)
sample_combined = [np.mean(df.sample(sample_size,replace=True)["sample"]) -\
                   np.mean(df.sample(sample_size, replace = True)["sample"]) for x in range(100000)]
sn.kdeplot(sample_combined, fill = True)
title = plt.title("Difference in the sample means")

## statistic for the sample in hands
plt.axvline(samp1.mean() - samp2.mean(), color = 'red')

## get p value
p_value = f"{sum(np.abs(sample_combined)>np.abs(samp1.mean() - samp2.mean()))/len(sample_combined):0.2%}"
print(p_value)